<a href="https://colab.research.google.com/github/sasansharifipour/Spark_Class/blob/main/train_sheypoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
file_download_link = "https://drive.google.com/u/0/uc?id=1K2a4mliN8l4aO1S8TWyn29EN25Y4WLlQ&export=download" 
!wget -O data.csv --no-check-certificate "$file_download_link"

--2021-05-22 04:13:18--  https://drive.google.com/u/0/uc?id=1K2a4mliN8l4aO1S8TWyn29EN25Y4WLlQ&export=download
Resolving drive.google.com (drive.google.com)... 64.233.189.102, 64.233.189.138, 64.233.189.100, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mcl2u08j9rve6as0bae6nihscmo2uhf6/1621656750000/14577798666193718332/*/1K2a4mliN8l4aO1S8TWyn29EN25Y4WLlQ?e=download [following]
--2021-05-22 04:13:19--  https://doc-0g-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mcl2u08j9rve6as0bae6nihscmo2uhf6/1621656750000/14577798666193718332/*/1K2a4mliN8l4aO1S8TWyn29EN25Y4WLlQ?e=download
Resolving doc-0g-70-docs.googleusercontent.com (doc-0g-70-docs.googleusercontent.com)... 108.177.97.132, 2404:6800:4008:c00::84
Connecting to doc-0g-70-docs.googleusercontent.com (do

In [2]:
import pandas as pd

data = pd.read_csv('data.csv')

df = pd.DataFrame(data)

In [22]:
df.head()

,Unnamed: 0,userid,time,source,adstatus,item,count_views
0,0,1274821301586/54e79450-bbee-4770-9e47-1d56e120...,2021-05-05 18:00:00.000,Chrome,۳ روز پیش,A,1
1,1,1274821301586/54e79450-bbee-4770-9e47-1d56e120...,2021-05-05 18:00:00.000,Chrome,۴ روز پیش,A,1
2,2,1325384907441/5e5766d8-46cb-480d-886a-7d557542...,2021-05-01 10:00:00.000,Android,۲ روز پیش,B,1
3,3,1325384907441/5e5766d8-46cb-480d-886a-7d557542...,2021-05-01 10:00:00.000,Android,۶ روز پیش,B,1
4,4,1337524499886/ad464731-6288-4e04-8a1a-aa9c004f...,2021-05-03 05:00:00.000,Android,ساعاتی پیش,A,1


In [46]:
import networkx as nx

def create_graph(data):
  graph = nx.Graph()

  for index, row in data.iterrows():
    source_node = row["userid"]
    destination_node = str(row["adstatus"]) + "_" + str(row["item"])
    prior_weight = row["count_views"]

    graph.add_node(source_node, node_type='user')
    graph.add_node(destination_node, node_type='item')

    node_attributes = graph.get_edge_data(source_node, destination_node)

    if (node_attributes is not None):
      prior_weight = prior_weight + node_attributes["weight"]

    graph.add_weighted_edges_from([(destination_node, source_node,  prior_weight)])

  return graph


graph = create_graph(df.head(10000))

In [47]:
page_ranks = nx.pagerank(graph, weight ='weight', max_iter =1000, alpha=0.9)

In [48]:
page_ranks

{'1274821301586/54e79450-bbee-4770-9e47-1d56e120bf80': 9.720725420539554e-05,
 '۳ روز پیش_A': 0.010028152303344673,
 '۴ روز پیش_A': 0.007523753485698146,
 '1325384907441/5e5766d8-46cb-480d-886a-7d5575421f84': 0.00012391847942822825,
 '۲ روز پیش_B': 0.00873291580232354,
 '۶ روز پیش_B': 0.003377463676776863,
 '1337524499886/ad464731-6288-4e04-8a1a-aa9c004fd88a': 0.0003892130910537937,
 'ساعاتی پیش_A': 0.14162862422561853,
 '۱ روز پیش_A': 0.029926502807872388,
 '۲ روز پیش_A': 0.0142842514064134,
 'دقایقی پیش_A': 0.026744320895328074,
 '۶ روز پیش_A': 0.0038460070284104747,
 'لحظاتی پیش_A': 0.02134602706091795,
 '1388595558952/140968f0-913f-455b-98b5-4ab1737ba2a4': 0.0003657665818502869,
 'ساعاتی پیش_C': 0.025980437284472156,
 'دقایقی پیش_C': 0.005730330095857518,
 '۲ روز پیش_C': 0.0039592099883455105,
 '۵ روز پیش_B': 0.003918452531803383,
 '۳ هفته پیش_C': 0.0018895202952684692,
 '۳ روز پیش_C': 0.0015189895527437686,
 '1388846967652/b95f5627-e7a8-4b2f-bace-6a8d4306f28e': 0.00141639205195265

In [ ]:
df[df.isnull().any(axis=1)]

,Unnamed: 0,userid,time,source,adstatus,item,count_views


In [ ]:
df["adstatus"].value_counts()

ساعاتی پیش    48208
۱ روز پیش     18756
۱ هفته پیش    15165
دقایقی پیش    13422
لحظاتی پیش    12156
۲ روز پیش     11864
۲ هفته پیش    10335
۳ روز پیش      8621
۳ هفته پیش     7908
۴ روز پیش      6951
۵ روز پیش      5864
۶ روز پیش      4939
۴ هفته پیش     2446
۱ ماه پیش      1104
۲ ماه پیش       688
۷ ماه پیش       264
۳ ماه پیش       232
۴ ماه پیش       202
۱ سال پیش       172
۵ ماه پیش        84
۶ ماه پیش        38
۱۱ ماه پیش       25
۱۰ ماه پیش       24
۸ ماه پیش        24
۹ ماه پیش        23
Name: adstatus, dtype: int64

In [ ]:
pd.get_dummies(df, columns=["item","adstatus", "source"]).head()

,Unnamed: 0,userid,time,count_views,item_A,item_B,item_C,adstatus_دقایقی پیش,adstatus_ساعاتی پیش,adstatus_لحظاتی پیش,adstatus_۱ روز پیش,adstatus_۱ سال پیش,adstatus_۱ ماه پیش,adstatus_۱ هفته پیش,adstatus_۱۰ ماه پیش,adstatus_۱۱ ماه پیش,adstatus_۲ روز پیش,adstatus_۲ ماه پیش,adstatus_۲ هفته پیش,adstatus_۳ روز پیش,adstatus_۳ ماه پیش,adstatus_۳ هفته پیش,adstatus_۴ روز پیش,adstatus_۴ ماه پیش,adstatus_۴ هفته پیش,adstatus_۵ روز پیش,adstatus_۵ ماه پیش,adstatus_۶ روز پیش,adstatus_۶ ماه پیش,adstatus_۷ ماه پیش,adstatus_۸ ماه پیش,adstatus_۹ ماه پیش,source_Android,source_Chrome
0,0,1274821301586/54e79450-bbee-4770-9e47-1d56e120...,2021-05-05 18:00:00.000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,1274821301586/54e79450-bbee-4770-9e47-1d56e120...,2021-05-05 18:00:00.000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,2,1325384907441/5e5766d8-46cb-480d-886a-7d557542...,2021-05-01 10:00:00.000,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,3,1325384907441/5e5766d8-46cb-480d-886a-7d557542...,2021-05-01 10:00:00.000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,4,1337524499886/ad464731-6288-4e04-8a1a-aa9c004f...,2021-05-03 05:00:00.000,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
